In [79]:
from __future__ import absolute_import, division, print_function

import collections
import numpy as np
from six.moves import range
import tensorflow as tf
import tensorflow_federated as tff
import tempfile
import os
import six

import json
import pandas as pd
import _pickle as pickle

tf.enable_resource_variables()

### Data preprocessing

In [21]:
##1. read raw data
bg1_path = 'data/20396154_entries.json'
bg2_path = 'data/99296581_entries.json'
df_bg1 = pd.read_json(bg1_path)
df_bg2 = pd.read_json(bg2_path)

In [115]:
##2. first step cleaning, remove unrelated information
def prefed_process(df_bg, client_id, window):
    df_bg = df_bg[['date', 'sgv']]    
    ret = df_bg.set_index('date').sort_index()
    in_start = 0
    fed = []
    for i in range(len(df_bg)):
        in_end = in_start + window
        if in_end < len(df_bg):
            fed.append((client_id, df_bg['sgv'][in_start:in_end].values.tolist()))
        in_start += 1 
    
    return fed

In [131]:
fed1 = prefed_process(df_bg1,'20396154',18)
fed2 = prefed_process(df_bg2,'99296581',18)

In [132]:
fed1[0]

('20396154',
 [137,
  143,
  148,
  149,
  147,
  149,
  146,
  138,
  131,
  130,
  127,
  121,
  116,
  109,
  107,
  105,
  108,
  112])

In [133]:
fed = fed1+fed2

In [185]:
def _create_example(features):
  """Convert a tuple of features to a tf.Example."""
  output_features = collections.OrderedDict()
  for i, feature in enumerate(features):
    if isinstance(feature, int):
      output_features[str(i)] = tf.train.Feature(
          int64_list=tf.train.Int64List(value=[feature]))
    elif isinstance(feature, float):
      output_features[str(i)] = tf.train.Feature(
          float_list=tf.train.FloatList(value=[feature]))
    elif isinstance(feature, list):
      output_features[str(i)] = tf.train.Feature(
          float_list=tf.train.FloatList(value=feature))
    else:
      # This is hit if the unittest is updated with unknown types, not an error
      # in the object under test. Extend the unittest capabilities to fix.
      raise NotImplementedError('Cannot handle feature type [%s]' %
                                type(feature))
  return tf.train.Example(features=tf.train.Features(
      feature=output_features)).SerializeToString()

In [186]:
class FakeUserData(object):
  """Container object that creates fake per-user data.
  Using the fake test data, create temporary per-user TFRecord files used for
  the test. Convert each feature-tuple to a `tf.Example` protocol buffer message
  and serialize it to the per-user file.
  """

  def __init__(self, test_data, temp_dir):
    """Construct a FakePerUseData object.
    Args:
      test_data: A list of tuples whose first element is the client ID and all
        subsequent elements are training example features.
      temp_dir: The path to the directory to store temporary per-user files.
    Returns:
      A dict of client IDs to string file paths to TFRecord files.
    """
    writers = {}
    client_file_dict = {}
    for example in test_data:
      client_id, features = example[0], example[1:]
      writer = writers.get(client_id)
      if writer is None:
        fd, path = tempfile.mkstemp(suffix=client_id, dir=temp_dir)
        # close the pre-opened file descriptor immediately to avoid leaking.
        os.close(fd)
        client_file_dict[client_id] = path
        writer = tf.python_io.TFRecordWriter(path=path)
        writers[client_id] = writer
      writer.write(_create_example(features))
    for writer in six.itervalues(writers):
      writer.close()
    self._client_data_file_dict = client_file_dict

  def create_test_dataset_fn(self, client_id):
    client_path = self._client_data_file_dict[client_id]
    features = {
#         '0': tf.FixedLenFeature(shape=[], dtype=tf.int64),
        '0': tf.FixedLenFeature(shape=[18], dtype=tf.float32),
    }

    def parse_example(e):
      feature_dict = tf.parse_single_example(serialized=e, features=features)
      return tuple(feature_dict[k] for k in sorted(six.iterkeys(feature_dict)))

    return tf.data.TFRecordDataset(client_path).map(parse_example)

  @property
  def client_ids(self):
    return list(self._client_data_file_dict.keys())


In [192]:
BATCH_SIZE = 128
SEQ_LENGTH = 18

BatchType = collections.namedtuple('BatchType', ['x', 'y'])

def split_input_target(chunk):
  input_text = tf.map_fn(lambda x: x[:12], chunk)
  target_text = tf.map_fn(lambda x: x[12:], chunk)
  return BatchType(input_text, target_text)


def preprocess(dataset):  
  return (dataset
      # each of length SEQ_LENGTH.
      .map(split_input_target))

In [193]:
temp_dir = tempfile.mkdtemp()
data = FakeUserData(fed, temp_dir)

In [195]:
example_data = preprocess(fed)

AttributeError: 'list' object has no attribute 'map'

In [ ]:
example = data.create_test_dataset_fn('20396154')

In [ ]:
example.take(1)

### Load Model

In [127]:
keras_conv_lstm = tf.keras.models.load_model('models/conv_lstm_128_2_6.kerasmodel', compile=False)

In [148]:
keras_conv_lstm.predict(np.array([137,143,148,149,147,149,146,138,131,130,127,121]).reshape(((1, 2, 1, 6, 1))))

array([[114.02403, 112.042  , 109.5221 , 109.03717, 108.37824, 107.89183]],
      dtype=float32)

In [135]:
def compile(keras_model):
  keras_model.compile(
      optimizer=tf.keras.optimizers.SGD(lr=0.5),loss='mse')
  return keras_model

In [177]:


def create_tff_model():
  # TFF uses a `dummy_batch` so it knows the types and shapes
  # that your model expects.
  x = tf.constant(np.random.random_sample((BATCH_SIZE, 2, 1, 6, 1))*200)
  x = tf.cast(x, 'float32')
  y = tf.constant(np.random.random_sample((BATCH_SIZE, 6))*200)
  y = tf.cast(y, 'float32')
  dummy_batch = collections.OrderedDict([('x', x), ('y', y)]) 
  keras_model_clone = compile(tf.keras.models.clone_model(keras_model))
  return tff.learning.from_compiled_keras_model(
      keras_model_clone, dummy_batch=dummy_batch)

In [178]:
fed_avg = tff.learning.build_federated_averaging_process(model_fn=create_tff_model)

W0620 16:28:02.401590 140439100831488 deprecation.py:323] From /home/jupyter/.local/lib/python3.5/site-packages/tensorflow_core/python/keras/optimizer_v2/optimizer_v2.py:460: BaseResourceVariable.constraint (from tensorflow.python.ops.resource_variable_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Apply a constraint manually following the optimizer update step.
W0620 16:28:02.421520 140439100831488 deprecation_wrapper.py:118] From /home/jupyter/.local/lib/python3.5/site-packages/tensorflow_federated/python/core/impl/tensorflow_serialization.py:296: The name tf.initializers.variables is deprecated. Please use tf.compat.v1.initializers.variables instead.

W0620 16:28:06.055349 140439100831488 deprecation.py:323] From /home/jupyter/.local/lib/python3.5/site-packages/tensorflow_core/python/ops/math_grad.py:1251: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instruc

In [197]:
fed[0]

('20396154',
 [137,
  143,
  148,
  149,
  147,
  149,
  146,
  138,
  131,
  130,
  127,
  121,
  116,
  109,
  107,
  105,
  108,
  112])

In [200]:
state = fed_avg.initialize()
state, metrics = fed_avg.next(state, ([]))
print(metrics)

TypeError: Unable to convert a Python object of type int into an `AnonymousTuple`.